# R

```
orto_request = function(polygon) {

  if (nrow(polygon) == 0) {
    stop("no geometries")
  }

  selected_cols = c("godlo", "akt_rok", "piksel", "kolor", "zrDanych", "ukladXY",
                    "czy_ark_wypelniony", "url_do_pobrania", "idSerie", "sha1",
                    "nazwa_pliku")
  selected_cols = paste(selected_cols, collapse = ",")

  epsg = sf::st_crs(polygon)$epsg

  # hard coded URL and parameters
  base_URL = "https://mapy.geoportal.gov.pl/gprest/services/SkorowidzeFOTOMF/MapServer/0/query?"
  geometryType = "&geometryType=esriGeometryEnvelope"
  spatialRel = "&spatialRel=esriSpatialRelIntersects"
  outFields = paste0("&outFields=", selected_cols)
  returnGeometry = "&returnGeometry=false"
  file = "&f=json"

  # initial empty df (columns must be identical as in 'selected_cols')
  empty_df = data.frame(godlo = character(),
                        akt_rok = integer(),
                        piksel = numeric(),
                        kolor = character(),
                        zrDanych = character(),
                        ukladXY = character(),
                        #modulArch = character(),
                        #nrZglosz = character(),
                        czy_ark_wypelniony = character(),
                        #daneAktualne = integer(),
                        #daneAktDo10cm = integer(),
                        #lok_orto = character(),
                        url_do_pobrania = character(),
                        idSerie = integer(),
                        sha1 = character(),
                        #akt_data = numeric(),
                        #idorto = integer(),
                        nazwa_pliku = character()
                        #ESRI_OID = integer()
  )

  for (i in seq_len(nrow(polygon))) {
    bbox = sf::st_bbox(sf::st_geometry(polygon)[[i]])

    # user input
    geometry = paste0("geometry={'xmin':", bbox[1], ", 'ymin':", bbox[2], ", ",
                      "'xmax':", bbox[3], ", 'ymax':", bbox[4], ", ",
                      "'spatialReference':{'wkid':", epsg, "}}")

    prepared_URL = paste0(base_URL, geometry, geometryType, spatialRel, outFields,
                          returnGeometry, file)

    output = jsonlite::fromJSON(prepared_URL)
    output = output$features[[1]]

    # MaxRecordCount: 1000
    if (nrow(output) == 1000) {
      warning("maximum number of records, reduce the area")
    }

    empty_df = rbind(empty_df, output)
  }

  # remove duplicated images
  empty_df = empty_df[!duplicated(empty_df$nazwa_pliku), ]

  # postprocessing
  colnames(empty_df) = c("sheetID", "year", "resolution", "composition",
                         "sensor", "CRS", "isFilled", "URL", "seriesID",
                         "sha1", "filename")
  empty_df$composition = as.factor(empty_df$composition)
  empty_df$CRS = as.factor(empty_df$CRS)
  empty_df$isFilled = ifelse(empty_df$isFilled == "TAK", TRUE, FALSE)
  empty_df$sensor = factor(empty_df$sensor,
             levels = c("Scena sat.", "Zdj. analogowe", "Zdj. cyfrowe"),
             labels = c("Satellite", "Analog", "Digital"))

  return(empty_df)
}
```

In [5]:
import requests

In [7]:
def orto_request(polygon):
    if len(polygon)==0:
        print("no geometries")
        return

    selected_cols_list = ["godlo", "akt_rok", "piksel", "kolor", "zrDanych", "ukladXY",
                    "czy_ark_wypelniony", "url_do_pobrania", "idSerie", "sha1",
                    "nazwa_pliku"]
    selected_cols = ','.join(selected_cols_list)

    #epsg = sf::st_crs(polygon)$epsg
    epsg="not-implemented-yet"

    # hard coded URL and parameters
    base_URL = "https://mapy.geoportal.gov.pl/gprest/services/SkorowidzeFOTOMF/MapServer/0/query?"
    geometryType = "&geometryType=esriGeometryEnvelope"
    spatialRel = "&spatialRel=esriSpatialRelIntersects"
    outFields = "&outFields="+selected_cols
    returnGeometry = "&returnGeometry=false"
    file = "&f=json"

#     # initial empty df (columns must be identical as in 'selected_cols')
#     empty_df = data.frame(godlo = character(),
#                         akt_rok = integer(),
#                         piksel = numeric(),
#                         kolor = character(),
#                         zrDanych = character(),
#                         ukladXY = character(),
#                         #modulArch = character(),
#                         #nrZglosz = character(),
#                         czy_ark_wypelniony = character(),
#                         #daneAktualne = integer(),
#                         #daneAktDo10cm = integer(),
#                         #lok_orto = character(),
#                         url_do_pobrania = character(),
#                         idSerie = integer(),
#                         sha1 = character(),
#                         #akt_data = numeric(),
#                         #idorto = integer(),
#                         nazwa_pliku = character()
#                         #ESRI_OID = integer()
#   )
    results=[]
    for i in range(len(polygon)):
#         bbox = sf::st_bbox(sf::st_geometry(polygon)[[i]])
        bbox = polygon[i]

        # user input
        geometry = ''.join([str(_) for _ in ["geometry={'xmin':", bbox[0], ", 'ymin':", bbox[1], ", ",
                          "'xmax':", bbox[2], ", 'ymax':", bbox[3], ", ",
                          "'spatialReference':{'wkid':", epsg, "}}"]])

        prepared_URL = ''.join([base_URL, geometry, geometryType, spatialRel, outFields,
                              returnGeometry, file])
        print("prepared_URL:",prepared_URL)
        
        r = requests.get(prepared_URL)
        print(r.status_code)
        if r.status_code==200:
            print(r.headers['content-type'])
            print(r.encoding)
            print(r.text)
            print(r.json())
            results.append(r.json())
#         output = jsonlite::fromJSON(prepared_URL)
#         output = output$features[[1]]

#         # MaxRecordCount: 1000
#         if (nrow(output) == 1000) {
#           warning("maximum number of records, reduce the area")
#         }

#         empty_df = rbind(empty_df, output)
#       }

#   # remove duplicated images
#   empty_df = empty_df[!duplicated(empty_df$nazwa_pliku), ]

#   # postprocessing
#   colnames(empty_df) = c("sheetID", "year", "resolution", "composition",
#                          "sensor", "CRS", "isFilled", "URL", "seriesID",
#                          "sha1", "filename")
#   empty_df$composition = as.factor(empty_df$composition)
#   empty_df$CRS = as.factor(empty_df$CRS)
#   empty_df$isFilled = ifelse(empty_df$isFilled == "TAK", TRUE, FALSE)
#   empty_df$sensor = factor(empty_df$sensor,
#              levels = c("Scena sat.", "Zdj. analogowe", "Zdj. cyfrowe"),
#              labels = c("Satellite", "Analog", "Digital"))

#   return(empty_df)
    return results


orto_request([[0,1,0,1],])

prepared_URL: https://mapy.geoportal.gov.pl/gprest/services/SkorowidzeFOTOMF/MapServer/0/query?geometry={'xmin':0, 'ymin':1, 'xmax':0, 'ymax':1, 'spatialReference':{'wkid':not-implemented-yet}}&geometryType=esriGeometryEnvelope&spatialRel=esriSpatialRelIntersects&outFields=godlo,akt_rok,piksel,kolor,zrDanych,ukladXY,czy_ark_wypelniony,url_do_pobrania,idSerie,sha1,nazwa_pliku&returnGeometry=false&f=json
200
application/json;charset=UTF-8
UTF-8
{"displayFieldName":"godlo","fieldAliases":{"godlo":"godlo","akt_rok":"akt_rok","piksel":"piksel","kolor":"kolor","zrDanych":"zrDanych","ukladXY":"ukladXY","czy_ark_wypelniony":"czy_ark_wypelniony","url_do_pobrania":"url_do_pobrania","idSerie":"idSerie","sha1":"sha1","nazwa_pliku":"nazwa_pliku"},"fields":[{"name":"godlo","type":"esriFieldTypeString","alias":"godlo","length":30},{"name":"akt_rok","type":"esriFieldTypeInteger","alias":"akt_rok"},{"name":"piksel","type":"esriFieldTypeDouble","alias":"piksel"},{"name":"kolor","type":"esriFieldTypeStri

[{'displayFieldName': 'godlo',
  'fieldAliases': {'godlo': 'godlo',
   'akt_rok': 'akt_rok',
   'piksel': 'piksel',
   'kolor': 'kolor',
   'zrDanych': 'zrDanych',
   'ukladXY': 'ukladXY',
   'czy_ark_wypelniony': 'czy_ark_wypelniony',
   'url_do_pobrania': 'url_do_pobrania',
   'idSerie': 'idSerie',
   'sha1': 'sha1',
   'nazwa_pliku': 'nazwa_pliku'},
  'fields': [{'name': 'godlo',
    'type': 'esriFieldTypeString',
    'alias': 'godlo',
    'length': 30},
   {'name': 'akt_rok', 'type': 'esriFieldTypeInteger', 'alias': 'akt_rok'},
   {'name': 'piksel', 'type': 'esriFieldTypeDouble', 'alias': 'piksel'},
   {'name': 'kolor',
    'type': 'esriFieldTypeString',
    'alias': 'kolor',
    'length': 10},
   {'name': 'zrDanych',
    'type': 'esriFieldTypeString',
    'alias': 'zrDanych',
    'length': 30},
   {'name': 'ukladXY',
    'type': 'esriFieldTypeString',
    'alias': 'ukladXY',
    'length': 10},
   {'name': 'czy_ark_wypelniony',
    'type': 'esriFieldTypeString',
    'alias': 'czy_a